In [1]:
import requests

In [2]:
from openai import OpenAI

In [3]:
import json

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
import os

In [7]:
stock_api = os.getenv("ALPHA_VANTAGE_API_KEY")

In [8]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [51]:
from agents import Agent, Runner, function_tool, ModelSettings, SQLiteSession, trace, RunConfig

In [10]:
from functools import lru_cache

In [11]:
from datetime import datetime, timedelta

In [12]:
url = "https://www.alphavantage.co/query?"

In [13]:
import json

In [14]:
@function_tool
def find_ticker(text: str) -> str:
    """Based on the natural language query, use this function to find the ticker symbol"""

    # params = {
    #     "function": "SYMBOL_SEARCH",
    #     "keywords": text,
    #     "apikey": stock_api
    # }

    # response = requests.get(url, params)

    # data = response.json()

    # ticker = (data.get('bestMatches')[0]).get('1. symbol')

    return _find_ticker(text)

In [15]:
@lru_cache(maxsize = 100)
def _find_ticker(text: str) -> str:
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": text,
        "apikey": stock_api
    }

    response = requests.get(url, params)

    data = response.json()

    ticker = data.get('bestMatches')

    filter_region = []

    for t in ticker:
        if t.get('4. region') == 'United States':
            filter_region.append(t)

    return filter_region[0].get('1. symbol')

In [16]:
_find_ticker('Microsoft')

'MSFT'

In [16]:
@function_tool
def get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    return _get_income_statement(ticker)

In [17]:
@lru_cache(maxsize = 100)
def _get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": stock_api
    }

    income_statement = requests.get(url,params)

    income_statement_data = income_statement.json()

    annual_data = income_statement_data.get('annualReports')

    quarterly_data = income_statement_data.get('quarterlyReports')

    filtered_data_annual = []

    filtered_data_quarterly = []

    for report in annual_data:
        filtered_data_annual.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    for report_quarterly in quarterly_data:
        filtered_data_quarterly.append({
            'fiscalDateEnding': report_quarterly.get('fiscalDateEnding'),
            'grossProfit': report_quarterly.get('grossProfit'),
            'totalRevenue': report_quarterly.get('totalRevenue'),
            'costOfRevenue': report_quarterly.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report_quarterly.get('costofGoodsAndServicesSold'),
            'operatingIncome': report_quarterly.get('operatingIncome'),
            'operatingExpenses': report_quarterly.get('operatingExpenses'),
            'researchAndDevelopment': report_quarterly.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report_quarterly.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report_quarterly.get('netInterestIncome'),
            'interestIncome': report_quarterly.get('interestIncome'),
            'depreciationAndAmortization': report_quarterly.get('depreciationAndAmortization'),
            'incomeBeforeTax': report_quarterly.get('incomeBeforeTax'),
            'incomeTaxExpense': report_quarterly.get('incomeTaxExpense'),
            'ebit': report_quarterly.get('ebit'),
            'ebitda': report_quarterly.get('ebitda'),
            'netIncome': report_quarterly.get('netIncome'),
        })

    final_data = {
        "annual_data": filtered_data_annual,
        "quarterly_data": filtered_data_quarterly[:8]
    }

    return final_data

In [206]:
_get_income_statement('TSLA')

{'annual_data': [{'fiscalDateEnding': '2024-12-31',
   'grossProfit': '17450000000',
   'totalRevenue': '97690000000',
   'costOfRevenue': '80240000000',
   'costofGoodsAndServicesSold': '80240000000',
   'operatingIncome': '7076000000',
   'operatingExpenses': '10374000000',
   'researchAndDevelopment': '4540000000',
   'sellingGeneralAndAdministrative': '5150000000',
   'netInterestIncome': '1219000000',
   'interestIncome': '1569000000',
   'depreciationAndAmortization': '5368000000',
   'incomeBeforeTax': '8990000000',
   'incomeTaxExpense': '1837000000',
   'ebit': '9340000000',
   'ebitda': '14708000000',
   'netIncome': '7130000000'},
  {'fiscalDateEnding': '2023-12-31',
   'grossProfit': '17660000000',
   'totalRevenue': '96773000000',
   'costOfRevenue': '79113000000',
   'costofGoodsAndServicesSold': '79113000000',
   'operatingIncome': '8891000000',
   'operatingExpenses': '8769000000',
   'researchAndDevelopment': '3969000000',
   'sellingGeneralAndAdministrative': '4800000

In [18]:
@function_tool
def get_annual_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement.
    """

    annual_filter = json_file.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    return filtered_data

In [101]:
get_annual_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2024-12-31',
  'grossProfit': '17450000000',
  'totalRevenue': '97690000000',
  'costOfRevenue': '80240000000',
  'costofGoodsAndServicesSold': '80240000000',
  'operatingIncome': '10374000000',
  'researchAndDevelopment': '4540000000',
  'sellingGeneralAndAdministrative': '5150000000',
  'netInterestIncome': '1219000000',
  'interestIncome': '1569000000',
  'depreciationAndAmortization': '5368000000',
  'incomeBeforeTax': '8990000000',
  'incomeTaxExpense': '1837000000',
  'ebit': '9340000000',
  'ebitda': '14708000000',
  'netIncome': '7130000000'},
 {'fiscalDateEnding': '2023-12-31',
  'grossProfit': '17660000000',
  'totalRevenue': '96773000000',
  'costOfRevenue': '79113000000',
  'costofGoodsAndServicesSold': '79113000000',
  'operatingIncome': '8769000000',
  'researchAndDevelopment': '3969000000',
  'sellingGeneralAndAdministrative': '4800000000',
  'netInterestIncome': '910000000',
  'interestIncome': '1066000000',
  'depreciationAndAmortization': '46670

In [19]:
@function_tool
def get_quarterly_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement datapoints.
    """

    quarterly_filter = json_file.get('quarterlyReports')

    filtered_qoq_data = []

    for report in quarterly_filter:
        filtered_qoq_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })


    return filtered_qoq_data[:4]

In [20]:
@function_tool
def get_balance_sheet(ticker: str) -> str:
    """Based on the ticker you can find the balance sheet of the company from this
    function. This will fetch both annual balance sheet only.
    """

    return _get_balance_sheet(ticker)

In [21]:
@lru_cache
def _get_balance_sheet(ticker: str) -> str:

    params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": stock_api
    }

    balance_sheet = requests.get(url,params)

    balance_sheet_data = balance_sheet.json()

    annual_data = balance_sheet_data.get('annualReports')

    quarterly_data = balance_sheet_data.get('quarterlyReports')

    filtered_data_annual = []

    filtered_data_quarterly = []

    for report in annual_data:
        filtered_data_annual.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
        })

    for report_quarterly in quarterly_data:
        filtered_data_quarterly.append({
            'fiscalDateEnding': report_quarterly.get('fiscalDateEnding'),
            'totalAssets': report_quarterly.get('totalAssets'),
            'totalCurrentAssets': report_quarterly.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report_quarterly.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report_quarterly.get('inventory'),
            'totalNonCurrentAssets': report_quarterly.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report_quarterly.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report_quarterly.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report_quarterly.get('totalLiabilities'),
            'interestIncome': report_quarterly.get('interestIncome'),
            'totalCurrentLiabilities': report_quarterly.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report_quarterly.get('currentLongTermDebt'),
            'totalShareholderEquity': report_quarterly.get('totalShareholderEquity'),
            'retainedEarnings': report_quarterly.get('retainedEarnings'),
        })


    final_data = {
        "annual_data": filtered_data_annual,
        "quarterly_data": filtered_data_quarterly[:8]
    }

    

    return final_data

In [201]:
_get_balance_sheet('TSLA')

{'annual_data': [{'fiscalDateEnding': '2024-12-31',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'inventory': '12017000000',
   'totalNonCurrentAssets': '63716000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'sellingGeneralAndAdministrative': None,
   'totalLiabilities': '48390000000',
   'interestIncome': None,
   'totalCurrentLiabilities': '28821000000',
   'currentLongTermDebt': '2343000000',
   'totalShareholderEquity': '72913000000',
   'retainedEarnings': '35209000000'},
  {'fiscalDateEnding': '2023-12-31',
   'totalAssets': '106618000000',
   'totalCurrentAssets': '49616000000',
   'cashAndCashEquivalentsAtCarryingValue': '16398000000',
   'inventory': '13626000000',
   'totalNonCurrentAssets': '57003000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'sellingGeneralAndAdministrative': None,
   'totalLiabilities': '43009000000',
   'interestIncome': None,
   't

In [119]:
cached_balance_sheet = get_balance_sheet(ticker_symbol)

cached_balance_sheet

{'symbol': 'TSLA',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'cashAndShortTermInvestments': '16139000000',
   'inventory': '12017000000',
   'currentNetReceivables': '4418000000',
   'totalNonCurrentAssets': '63716000000',
   'propertyPlantEquipment': '51501000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': '1226000000',
   'intangibleAssetsExcludingGoodwill': '1226000000',
   'goodwill': '244000000',
   'investments': 'None',
   'longTermInvestments': 'None',
   'shortTermInvestments': '20424000000',
   'otherCurrentAssets': '5362000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '48390000000',
   'totalCurrentLiabilities': '28821000000',
   'currentAccountsPayable': '12474000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '32

In [22]:
@function_tool
def annual_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding annual balance sheet datapoints.
    """

    annual_filter = json_data.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [23]:
@function_tool
def quarterly_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding quarterly balance sheet datapoints.
    """

    quarterly_filter = json_data.get('quarterlyReports')

    filtered_data = []

    for report in quarterly_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [24]:
@function_tool
def cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    return _cash_flow_statement(ticker)

In [25]:
@lru_cache
def _cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    params = {
        "function": "CASH_FLOW",
        "symbol": ticker,
        "apikey": stock_api
    }

    cash_flow = requests.get(url,params)

    cash_flow_data = cash_flow.json()

    annual_filter = cash_flow_data.get('annualReports')

    final_cash_flow_data = []

    for cash_flow in annual_filter:
        final_cash_flow_data.append({
            'fiscalDateEnding': cash_flow.get('fiscalDateEnding'),
            'operatingCashflow': cash_flow.get('operatingCashflow'),
            'cashflowFromInvestment': cash_flow.get('cashflowFromInvestment'),
            'cashflowFromFinancing': cash_flow.get('cashflowFromFinancing'),
            'netIncome': cash_flow.get('netIncome')
        })

    return final_cash_flow_data

In [134]:
cash_flow_statement(ticker_symbol)

[{'fiscalDateEnding': '2024-12-31',
  'operatingCashflow': '14923000000',
  'cashflowFromInvestment': '-18787000000',
  'cashflowFromFinancing': '3853000000',
  'netIncome': '7153000000'},
 {'fiscalDateEnding': '2023-12-31',
  'operatingCashflow': '13256000000',
  'cashflowFromInvestment': '-15584000000',
  'cashflowFromFinancing': '2589000000',
  'netIncome': '14974000000'},
 {'fiscalDateEnding': '2022-12-31',
  'operatingCashflow': '14724000000',
  'cashflowFromInvestment': '-11973000000',
  'cashflowFromFinancing': '-3527000000',
  'netIncome': '12556000000'},
 {'fiscalDateEnding': '2021-12-31',
  'operatingCashflow': '11497000000',
  'cashflowFromInvestment': '-7868000000',
  'cashflowFromFinancing': '-5203000000',
  'netIncome': '5519000000'},
 {'fiscalDateEnding': '2020-12-31',
  'operatingCashflow': '5943000000',
  'cashflowFromInvestment': '-3132000000',
  'cashflowFromFinancing': '9973000000',
  'netIncome': '690000000'},
 {'fiscalDateEnding': '2019-12-31',
  'operatingCashflo

In [26]:
@function_tool
def price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    return _price_data(ticker)

In [27]:
@lru_cache
def _price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "apikey": stock_api
    }

    response = requests.get(url,params)

    response_json = response.json()

    filtered_json = response_json['Time Series (Daily)']

    close_price = []

    for date, values in filtered_json.items():
        close_price.append(
            {
                'date': date,
                'close': values['4. close']
            }
        )

    latest_price = close_price[0]

    return latest_price

In [28]:
@function_tool
def earnings_analysis(ticker: str) -> str:
    """ 
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    return _earnings_analysis(ticker)

In [29]:
@lru_cache
def _earnings_analysis(ticker: str) -> str:
    """
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    final_transcript = []

    final_quarter = []

    while final_transcript == []:
        today = datetime.now()
    
        year = today.year
        month = today.month
    
        if month in [1,2,3]:
            quarter = 'Q4'
            year -= 1
        elif month in [4,5,6]:
            quarter = 'Q1'
        elif month in [7,8,9]:
            quarter = 'Q2'
        else:
            quarter = 'Q3'
    
        final = f'{year}{quarter}'
    
        params = {
            "function": "EARNINGS_CALL_TRANSCRIPT",
            "symbol": ticker,
            "apikey": stock_api,
            "quarter": final
        }

        response = requests.get(url, params)

        final_json = response.json()

        transcript = final_json.get('transcript')

        quarter = final_json.get('quarter')

        final_transcript.append(transcript)

        final_quarter.append(quarter)

    
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
           You will be given the full earnings call transcript in {final_transcript}.

            Your task is to extract **7–10 investor-relevant insights** strictly based on what 
            **Apple management said** in the transcript.
            
            Rules you MUST follow:
            1. Use ONLY information explicitly stated by Apple executives (CEO, CFO, or Investor Relations).  
               Do NOT include analyst opinions, assumptions, or external interpretation.
            2. Each point must represent a **distinct insight**, not a restatement of another point.
            3. For every point, clearly mention **who said it** (name + role).
            4. Do NOT use generic phrases such as “strong performance”, “positive outlook”, or “solid results” 
            unless backed by specific data or commentary.
            5. Each point must explain **why this matters to a long-term investor** 
            (growth, margins, cash flow, competitive position, or strategic direction).
            6. Prefer **synthesis over repetition** — combine numbers with management commentary where relevant.
            7. Do NOT introduce any facts, estimates, or forward views not explicitly stated in the transcript.
            
            Output format:
            - Heading: Analysis of the earnings call ({final_quarter})
            - Numbered list (7–10 points)
            - 2–4 sentences per point
            - End each point with: *(Source: Name, Title)*
            
            Tone:
            - Analytical
            - Investor-oriented
            - Neutral and factual

            Don't only include positive or negative points, try to include both
            """}
        ], temperature = 0.3
    )

    return response.choices[0].message.content

In [172]:
_earnings_analysis('MSFT')

"# Analysis of the earnings call (['2025Q3'])\n\n1. Microsoft Cloud revenue surpassed $42 billion, reflecting a 22% growth in constant currency. This significant increase underscores the company's strong position in the cloud market, which is critical for long-term growth as businesses increasingly migrate to cloud solutions. The robust demand for cloud services indicates that Microsoft is effectively capturing market share and expanding its customer base. *(Source: Satya Nadella, CEO)*\n\n2. The company reported a commercial bookings increase of 18%, driven by Azure commitments from major clients like OpenAI. This growth in bookings is a positive indicator of future revenue, as it suggests that customers are making long-term commitments to Microsoft's cloud services, which can enhance revenue predictability and stability. *(Source: Amy Hood, CFO)*\n\n3. Microsoft’s commercial remaining performance obligation rose to $315 billion, up 34% year-over-year. This metric is crucial as it ref

In [45]:
# global_session = SQLiteSession("test_conversation")

In [54]:
async def _agent_builder(query, workflow = "random_test"):

    agent = Agent(
        name = "Data Finder",
        model = 'gpt-4o-mini',
        instructions = f"""
        You are a stock analyser and your job is to understand the {query} and then use the tools available at your disposal 
        to answer to the user. 

        Make sure that you ONLY get data from tools and in case the query cannot be solved by tools, empathetically refuse to answer
        the user query. 

        It is recommended you use tables, if you're doing any comparison either between 2 or more companies or you're 
        showing the data of more than 1 year.

        In the tools you are now getting annual and quarterly information for {get_income_statement} & {get_balance_sheet}.

        You can use both to do your analysis now. 
        
        Quarterly data is for the last 8 quarters!

        You can use the {earnings_analysis} tool in case the user wants to know what happened in the latest earnings call of a particular
        company.

        However, you should always try to back it up with financial data first. 
        
        EXCEPTION:
        If the user is specifically asking to summarise the earning call data you are supposed to simply just print out
        the output you're getting from {earnings_analysis}.
        """,
        tools = [
            price_data,
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement,
            earnings_analysis
        ],
        model_settings = ModelSettings(
            tool_choice = "auto",
            seed = 0,
            temperature = 0.3
        )
    )
    
    result = await Runner.run(agent, query, run_config = RunConfig(workflow_name = workflow))

    return result

    # tools_called = {
    #     item.raw_item.name
    #     for item in result.new_items

    #     if item.type == "tool_call_item"
    # }
    # return tools_called

In [55]:
await _agent_builder("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check")

RunResult(input='Does google have a greater revenue than microsoft?', new_items=[ToolCallItem(agent=Agent(name='Data Finder', handoff_description=None, tools=[FunctionTool(name='price_data', description='Based on the ticker, we need to find the price of the stock through this.', params_json_schema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'price_data_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1153ddd00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='cash_flow_statement', description='Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this\nfunction. This will fetch both quarterly and annual cash flow statement.', params_json_schema={'properties': {'ticker': {'title': 'Ticker', 'ty

In [211]:
#testing quarterly data

await _agent_builder("Okay, give me a breakdown of their latest income & expenses")

Here’s a detailed breakdown of Google’s (Alphabet Inc.) latest quarterly income and expenses (Q3 2025):

| Item                               | Amount (USD)      |
|-------------------------------------|-------------------|
| **Total Revenue**                   | $102,346,000,000  |
| **Cost of Revenue**                 | $41,369,000,000   |
| **Gross Profit**                    | $60,977,000,000   |
| **Operating Expenses**              | $29,749,000,000   |
| - Research & Development            | $15,151,000,000   |
| - Selling, General & Administrative | $7,393,000,000    |
| **Operating Income**                | $31,228,000,000   |
| **Net Interest Income**             | $933,000,000      |
| **Depreciation & Amortization**     | $5,611,000,000    |
| **Income Before Tax**               | $43,987,000,000   |
| **Income Tax Expense**              | $9,008,000,000    |
| **Net Income**                      | $34,979,000,000   |
| **EBITDA**                          | $49,741,000,000 

In [184]:
#testing earning analysis

await _agent_builder("What happened in the latest earnings call of microsoft?")

Here’s a summary of what happened in the latest Microsoft earnings call (2025 Q3):

1. Microsoft Cloud surpassed $42 billion in revenue, growing 22% in constant currency.
2. Commercial bookings increased by 18%, driven by Azure commitments from OpenAI.
3. Microsoft’s AI business revenue exceeded expectations, with Azure AI services contributing significantly to growth.
4. The gross margin percentage for Microsoft Cloud was 69%, down three points year-over-year due to scaling AI infrastructure.
5. Microsoft 365 commercial cloud revenue grew 12%, driven by increased adoption of E5 and M365 Copilot.
6. Capital expenditures for the quarter were $21.4 billion, slightly lower than expected, showing efficiency in data center build-outs.
7. Commercial remaining performance obligation reached $315 billion, up 34% year-over-year.
8. Paid Microsoft 365 commercial seats grew 7% year-over-year to over 430 million.
9. AI services led to a 16-point growth rate lift to Azure.
10. Microsoft is continuo

In [181]:
await _agent_builder("Give an overall review of Google, are they on the right track?")

Here’s an overall review of Google (Alphabet Inc.) based on the latest financial and qualitative data:

### Financial Performance (2024 vs. Previous Years)

| Year  | Total Revenue | Net Income   | Operating Income | Free Cash Flow (TTM) |
|-------|--------------|--------------|------------------|----------------------|
| 2024  | $350.0B      | $100.1B      | $112.4B          | $73.6B               |
| 2023  | $307.4B      | $73.8B       | $84.3B           | —                    |
| 2022  | $282.8B      | $60.0B       | $74.8B           | —                    |

- **Revenue and profit are at all-time highs,** with 2024 marking Alphabet’s first-ever $100 billion quarter.
- **Operating margins** remain strong (30.5%), indicating efficient cost management.

### Strategic & Operational Highlights

- **AI Leadership:** Alphabet’s Gemini AI app has over 650 million monthly active users, and AI is deeply integrated into products like Search and YouTube.
- **Cloud Growth:** Google Cloud revenu

In [79]:
#testing session settings
await _agent_builder("I want income details related to Alphabet Inc?")

Here are the key income details for Alphabet Inc. (GOOGL) for the past 5 years:

| Fiscal Year | Total Revenue | Gross Profit | Operating Income | Net Income   | EBITDA        |
|-------------|--------------|--------------|------------------|--------------|--------------|
| 2024        | $350.02B     | $203.71B     | $112.39B         | $100.12B     | $135.39B     |
| 2023        | $307.39B     | $174.06B     | $84.29B          | $73.80B      | $97.97B      |
| 2022        | $282.84B     | $156.63B     | $74.84B          | $59.97B      | $85.16B      |
| 2021        | $257.64B     | $146.70B     | $78.71B          | $76.03B      | $103.52B     |
| 2020        | $182.53B     | $97.80B      | $41.22B          | $40.27B      | $61.91B      |

**Key Terms:**
- **Total Revenue:** All income from sales and services.
- **Gross Profit:** Revenue minus cost of goods/services sold.
- **Operating Income:** Profit from core business operations.
- **Net Income:** Profit after all expenses, taxes, an

In [80]:
#testing session settings
await _agent_builder("Can you give me a detailed breakdown of R&D expenses?")

Here is a detailed breakdown of Alphabet Inc.'s (GOOGL) Research & Development (R&D) expenses for the past 5 years:

| Fiscal Year | R&D Expenses (USD) |
|-------------|--------------------|
| 2024        | $49,326,000,000    |
| 2023        | $45,427,000,000    |
| 2022        | $39,500,000,000    |
| 2021        | $31,562,000,000    |
| 2020        | $27,573,000,000    |

**Observations:**
- Alphabet’s R&D spending has consistently increased year-over-year.
- In 2024, R&D expenses reached nearly $50 billion, highlighting Alphabet’s strong focus on innovation and technology development.

If you want a longer historical view or a comparison with other companies, just let me know!


In [68]:
#testing session settings
await _agent_builder("Do you think it is justified to spend so much on R&D?")

I understand your interest in whether Alphabet Inc.'s high R&D spending is justified. However, as an AI, I can only provide factual data and cannot offer personal opinions or subjective analysis. 

If you would like, I can provide you with:
- R&D as a percentage of total revenue over the years
- Comparison of Alphabet’s R&D spending with other major tech companies

Let me know how you’d like to proceed!


In [69]:
#testing session settings
await _agent_builder("Why are they spending so much on R&D? Is it their largest expense?")

Let's analyze Alphabet Inc.'s (GOOGL) expenses to answer your questions:

### 1. Why is Alphabet spending so much on R&D?
Alphabet is a technology-driven company whose core businesses (Google Search, YouTube, Android, Cloud, AI, etc.) rely heavily on innovation. High R&D spending allows Alphabet to:
- Maintain leadership in search, advertising, and cloud services
- Develop new products and services (e.g., AI, self-driving cars, quantum computing)
- Stay competitive against other tech giants

### 2. Is R&D their largest expense?
Let's compare R&D expenses with other major expense categories for the last 3 years:

| Fiscal Year | R&D Expense | Cost of Revenue | SG&A Expense | Operating Expenses (Total) |
|-------------|-------------|-----------------|--------------|---------------------------|
| 2024        | $49.33B     | $146.31B        | $14.19B      | $91.32B                   |
| 2023        | $45.43B     | $133.33B        | $16.43B      | $89.77B                   |
| 2022        |

In [54]:
await agent_builder("Give me details about the tesla stock?")

"Tesla, Inc. (ticker: TSLA) has been a prominent player in the electric vehicle market, and as of December 24, 2025, its stock price closed at $485.40. Let’s delve into some key financial metrics to better understand its current standing.\n\nFor the fiscal year ending December 31, 2024, Tesla reported total revenue of approximately $97.69 billion, showcasing robust growth. The company achieved a gross profit of $17.45 billion, indicating a healthy gross margin. However, the cost of revenue was significant at $80.24 billion, which highlights the challenges of scaling production and maintaining profitability.\n\nOperating income stood at $7.08 billion, with operating expenses totaling $10.37 billion, reflecting substantial investments in research and development ($4.54 billion) and selling, general, and administrative costs ($5.15 billion). The net income for the year was $7.13 billion, after accounting for an income tax expense of $1.84 billion.\n\nOn the balance sheet, Tesla had total 

In [93]:
await _agent_builder("Give me details about the tesla stock?")

Here are the latest details about Tesla (TSLA) stock, including its price, key financials, and recent performance:

---

### Tesla (TSLA) Stock Snapshot

| Metric                | Value (Latest)      |
|-----------------------|---------------------|
| Stock Price (as of 2025-12-24) | $485.40            |
| Fiscal Year End       | December 31, 2024   |

---

### Key Financials (Annual)

#### Income Statement

| Year | Total Revenue | Net Income | Gross Profit | Operating Income | EBITDA      |
|------|--------------|------------|--------------|------------------|-------------|
| 2024 | $97.69B      | $7.13B     | $17.45B      | $7.08B           | $14.71B     |
| 2023 | $96.77B      | $14.97B    | $17.66B      | $8.89B           | $14.80B     |
| 2022 | $81.46B      | $12.59B    | $20.85B      | $13.66B          | $17.83B     |

#### Balance Sheet

| Year | Total Assets | Total Liabilities | Shareholder Equity | Cash & Equivalents |
|------|-------------|-------------------|-------------

In [95]:
#gpt 4.1
await _agent_builder("Help me understand the financial performance of Alphabet Inc, is it a good buy?")

Here’s a comprehensive analysis of Alphabet Inc.’s (GOOGL) financial performance based on the latest available data:

### 1. Income Statement (Key Metrics, USD Millions)

| Year      | Revenue     | Net Income | Gross Profit | Operating Income | EPS (if available) |
|-----------|-------------|------------|--------------|------------------|--------------------|
| 2024      | 350,018     | 100,118    | 203,712      | 112,390          | N/A                |
| 2023      | 307,394     | 73,795     | 174,062      | 84,293           | N/A                |
| 2022      | 282,836     | 59,972     | 156,633      | 74,842           | N/A                |
| 2021      | 257,637     | 76,033     | 146,698      | 78,714           | N/A                |
| 2020      | 182,527     | 40,269     | 97,795       | 41,224           | N/A                |

**Observations:**
- Revenue and net income have shown strong growth, especially in the last two years.
- Operating income and gross profit margins are robus

In [96]:
#gpt-4.1
await _agent_builder("Compare the performance of Oracle and Salesforce, and choose which one is a better bet?")

Here is a comprehensive comparison of Oracle (ORCL) and Salesforce (CRM) based on their latest financial performance, balance sheets, and cash flows. This should help determine which company is a better bet as of the most recent fiscal years.

---

### Stock Price (as of 2025-12-24)
| Company    | Ticker | Price (USD) |
|------------|--------|-------------|
| Oracle     | ORCL   | 197.49      |
| Salesforce | CRM    | 265.26      |

---

### Income Statement (Most Recent Fiscal Year)
| Metric                | Oracle (2025)      | Salesforce (2025)  |
|-----------------------|--------------------|--------------------|
| Total Revenue         | $57.40B            | $37.90B            |
| Gross Profit          | $40.47B            | $29.25B            |
| Operating Income      | $17.68B            | $7.21B             |
| Net Income            | $12.44B            | $6.20B             |
| EBITDA                | $23.91B            | $11.14B            |

---

### Balance Sheet (Most Recen

In [97]:
#gpt 4.1
await _agent_builder("Help me understand the revenue of Alphabet Inc, is it growing year over year?")

Here is a summary of Alphabet Inc.'s (Google's) annual revenue for the past several years:

| Year | Total Revenue (USD) |
|------|---------------------|
| 2024 | $350,018,000,000    |
| 2023 | $307,394,000,000    |
| 2022 | $282,836,000,000    |
| 2021 | $257,637,000,000    |
| 2020 | $182,527,000,000    |
| 2019 | $161,857,000,000    |
| 2018 | $136,819,000,000    |
| 2017 | $110,855,000,000    |
| 2016 | $90,272,000,000     |
| 2015 | $74,989,000,000     |

**Analysis:**
- Alphabet Inc.'s revenue has consistently grown year over year.
- From 2020 to 2024, revenue increased from $182.5 billion to $350.0 billion.
- The growth trend is strong, with each year showing a significant increase over the previous year.

**Conclusion:**  
Yes, Alphabet Inc.'s revenue is growing year over year, and the growth rate has been robust, especially in recent years. If you need a more detailed analysis or growth rate calculation, let me know!


In [98]:
#gpt-4.1
await _agent_builder("Compare the performance of Oracle and Salesforce, and choose which one is a better bet?")

Here’s a detailed comparison of Oracle (ORCL) and Salesforce (CRM) based on their latest financials, stock price, and key performance metrics. All data is from their most recent annual filings.

---

### 1. Stock Price (as of Dec 24, 2025)

| Company    | Stock Price (USD) |
|------------|-------------------|
| Oracle     | $197.49           |
| Salesforce | $265.26           |

---

### 2. Income Statement (FY 2025)

| Metric              | Oracle (ORCL)      | Salesforce (CRM)   |
|---------------------|--------------------|--------------------|
| Total Revenue       | $57.40B            | $37.90B            |
| Gross Profit        | $40.47B            | $29.25B            |
| Operating Income    | $17.68B            | $7.21B             |
| Net Income          | $12.44B            | $6.20B             |
| EBITDA              | $23.91B            | $11.14B            |
| Net Margin          | 21.7%              | 16.3%              |

---

### 3. Balance Sheet (FY 2025)

| Metric    

In [32]:
#gpt 4.1
await _agent_builder("Help me understand the financial performance of Alphabet Inc, is it a good buy?")

Here’s a comprehensive analysis of Alphabet Inc.’s (GOOGL) financial performance and whether it could be considered a good buy, based on the latest available data:

---

## 1. Financial Performance Overview (Annual)

| Year         | Revenue ($B) | Net Income ($B) | Gross Profit ($B) | Operating Income ($B) | R&D Expense ($B) |
|--------------|--------------|-----------------|-------------------|----------------------|------------------|
| 2024 (TTM)   | 350.0        | 100.1           | 203.7             | 112.4                | 49.3             |
| 2023         | 307.4        | 73.8            | 174.1             | 84.3                 | 45.4             |
| 2022         | 282.8        | 59.9            | 156.6             | 74.8                 | 39.5             |
| 2021         | 257.6        | 76.0            | 146.7             | 78.7                 | 31.6             |
| 2020         | 182.5        | 40.3            | 97.8              | 41.2                 | 27.6             

In [33]:
await _agent_builder("Latest quarterly earnings of Google?")

Here is a summary of Alphabet Inc. (Google)'s latest quarterly (Q3 2025) earnings performance:

---

## Alphabet Inc. (GOOGL) – Q3 2025 Earnings Highlights

| Metric                  | Q3 2025 (Latest)   |
|-------------------------|--------------------|
| **Total Revenue**       | $102.35 Billion    |
| **Gross Profit**        | $60.98 Billion     |
| **Operating Income**    | $31.23 Billion     |
| **Operating Margin**    | 30.5%              |
| **Net Income**          | $34.98 Billion     |
| **Free Cash Flow**      | $24.5 Billion      |
| **Google Cloud Revenue**| $15.2 Billion      |
| **Paid Subscriptions**  | >$300 Million      |

---

### Key Takeaways from the Earnings Call

1. **Record Revenue:** Alphabet delivered its first-ever $100 billion quarter, doubling revenue from five years ago.
2. **AI-Driven Growth:** The Gemini app now has over 650 million monthly active users, with AI queries tripling from Q2.
3. **Cloud Segment Expansion:** Google Cloud revenue grew 34% YoY, 

In [43]:
with trace("Annual Income Statement Call"):
    first_result = await _agent_builder("Give me a brief about the revenue of Google?")
    # second_result = await _agent_builder("Give me the latest revenue data of Google")

    # print(first_result)
    # print("--------")
    # print(second_result)

    print(first_result)

RunResult:
- Last agent: Agent(name="Data Finder", ...)
- Final output (str):
    Here's a brief overview of Alphabet Inc. (Google)'s revenue:
    
    ### Alphabet Inc. (Google) Revenue Overview
    
    #### Annual Revenue (Last Three Years)
    
    - **2024 (Projected):** **$350 billion**
    - **2023:** **$307.4 billion**
    - **2022:** **$282.8 billion**
    
    **Growth Trends:**
    - Revenue is expected to grow significantly in 2024, reflecting a strong focus on AI and cloud services.
    - The increase from 2022 to 2023 shows a steady growth rate of approximately **8.5%**.
    
    #### Quarterly Revenue (Latest Quarter)
    
    - **Q3 2025:** **$102.35 billion** (first quarter to exceed $100 billion)
    - **Q2 2025:** **$96.4 billion**
    - **Q1 2025:** **$90.2 billion**
    
    ### Key Revenue Drivers
    
    1. **Advertising Revenue:** The primary source of income, benefiting from increased digital ad spending.
    2. **Google Cloud:** Significant growth, contributi